In [1]:
import cobra

In [2]:
pichia = cobra.io.read_sbml_model("data/iLC915_rewritten.xml")

When we simulate the model, we get a growth rate of $0.08 h^{-1}$

In [3]:
pichia.optimize()

,fluxes,reduced_costs
EX_m1282,0.000000,0.000000e+00
EX_m1283,0.000000,0.000000e+00
EX_m1284,0.000000,0.000000e+00
EX_m1285,0.000000,0.000000e+00
EX_m1286,0.000000,0.000000e+00
...,...,...
r1188,0.000000,-1.296205e-02
r1998,0.000000,-8.985868e-16
r1996,0.000000,0.000000e+00
r1997,0.000000,-1.147084e-01


Unfortunately, the model uses just numerical IDs (which is very common) instead of human readable IDs, which will make working with it hard. Using the `summary` function we see that one metabolite is taken up with a flux of $1 mmol gDW^{-1} h^{-1}$ (those are the default units in genome-scale metabolic models).

In [4]:
pichia.reactions.r1145.bounds=(0,1)

In [5]:
pichia.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m1293,EX_m1293,1,0,0.00%
m1298,EX_m1298,0.7968,0,0.00%
m1300,EX_m1300,0.4628,0,0.00%
m1301,EX_m1301,0.03308,0,0.00%
m1304,EX_m1304,0.00246,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
m1287,EX_m1287,-0.0803,0,0.00%
m1291,EX_m1291,-1.597,0,0.00%
m1349,EX_m1349,-3.432,0,0.00%


And that metabolite is glucose.

In [5]:
pichia.reactions.EX_m1293

Reaction identifier,EX_m1293
Name,EX_m1293
Memory address,0x0226aa418518
Stoichiometry,m1293 <=> alpha-D-Glucose_C6H12O6 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [10]:
pichia.metabolites.m1293.reactions

frozenset({<Reaction EX_m1293 at 0x27870c99da0>,
           <Reaction r1145 at 0x2787152d748>})

Looks like the medium is not defined at the exchange reaction level (i.e. reactions starting with `EX_`) but rather on the transport reaction level. The upper bound of reaction r1145 is 1.0 which means that the model can take up glucose < 1 mmol gDW^-1 h^-1.

In [11]:
pichia.reactions.r1145

Reaction identifier,r1145
Name,Uptake of alpha-D-Glucose
Memory address,0x02787152d748
Stoichiometry,m1293 --> m2 alpha-D-Glucose_C6H12O6 --> alpha-D-Glucose_C6H12O6
GPR,
Lower bound,0.0
Upper bound,1.0


Let's change the medium to methanol. Need to eliminate glucose uptake and enable methanol uptake. Let's make a copy of the model first though and also find the methanol transport reactions.

In [101]:
pichia.reactions.query("methanol", "name")

[<Reaction r1158 at 0x2787153c320>]

In [103]:
pichia.reactions.EX_m1297

Reaction identifier,EX_m1297
Name,EX_m1297
Memory address,0x027870c9b080
Stoichiometry,m1297 <=> Methanol_CH4O <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


This reaction is the uptake of methanol, and as it was said above in this model the medium is not defined at the exchange reaction level (EX_ reactions), but at the transport level, which should be this reaction. 

IMPORTANT!!: Notice that the upper and lower limits have been set to 0, thus we are forcing our system to not use any methanol. 

Let's get further details and check the methanol species involved, as well as other reactions because by typing "Methanol" with capital M on the query, 3 other reactions appear:

In [18]:
pichia.reactions.query("Methanol", "name")

[<Reaction r91 at 0x2787139ff98>,
 <Reaction r1310 at 0x2787146ea58>,
 <Reaction r1382 at 0x278714fbf60>]

In [58]:
print(pichia.reactions.r91, pichia.reactions.r1310, pichia.reactions.r1382)
print('r91', pichia.reactions.r91.name, ':', pichia.metabolites.m711.name, '+',  pichia.metabolites.m956.name, '-->',  pichia.metabolites.m713.name, '+',
     pichia.metabolites.m957.name)
print('r1310',  pichia.reactions.r1310.name, ':', pichia.metabolites.m1215.name, '-->',  pichia.metabolites.m956.name )
print('r1382',  pichia.reactions.r1382.name, ':', pichia.metabolites.m1219.name, '-->',  pichia.metabolites.m1215.name )

r91: m711 + m956 --> m713 + m957 r1310: m1215 --> m956 r1382: m1219 --> m1215
r91 Methanol:oxygen oxidereductase : Oxygen_O2 + Methanol_CH4O --> H2O2_H2O2 + Formaldehyde_CH2O
r1310 Methanol peroxisomal transport : Methanol_CH4O --> Methanol_CH4O
r1382 Methanol permease : Methanol_CH4O --> Methanol_CH4O


Therefore, clearly r91 is the the start of the MUT pathway, and then presumably the order for the others is r1158 with the uptake of methanol (m1297 --> m1219) that then goes to the methanol permease reaction r1382 (m1219 --> m1215), that goes into the peroxisomal transport r1310 (m1215 --> m956), which then together with oxygen (m711), starts MUT pathway.

In [63]:
pichia.metabolites.m1297

Metabolite identifier,m1297
Name,Methanol_CH4O
Memory address,0x02787039fac8
Formula,
Compartment,C_b
In 2 reaction(s),"EX_m1297, r1158"


Indeed, this is the one as the m1297 methanol place on the compartment `C_b`, meaning boundary, is involved on the uptake reaction r1158, as we saw, as well as an `EX_` reaction.

In [64]:
pichia.reactions.EX_m1297

Reaction identifier,EX_m1297
Name,EX_m1297
Memory address,0x027870c9b080
Stoichiometry,m1297 <=> Methanol_CH4O <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [9]:
pichia_methanol = pichia.copy()
pichia_methanol.reactions.r1145.bounds = 0, 0  # no glucose 
pichia_methanol.reactions.r1158.bounds = 0, 1 # add methanol to the medium with a maxium uptake of 10 mmol gDW^-1 h^-1
# there was a problem when declaring the bounds, as the last "bounds" part was not added, thus all the reaction r1158, was just 
# changed to (0,10), that is why all the model was failing or not growing, because I guess neither glucose nor methanol was
# available for K. phaffii

In [7]:
pichia_methanol.reactions.r1158

Reaction identifier,r1158
Name,uptake of methanol
Memory address,0x0226ab4c8668
Stoichiometry,m1297 --> m1219 Methanol_CH4O --> Methanol_CH4O
GPR,
Lower bound,0
Upper bound,10


(Solved!) Looks like growth is not supported when glucose is swapped with methanol. Something needs to be added. 

Remember m1293 is glucose and m1297 is methanol

In [10]:
pichia_methanol.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m1297,EX_m1297,1,0,0.00%
m1298,EX_m1298,1.258,0,0.00%
m1300,EX_m1300,0.06602,0,0.00%
m1301,EX_m1301,0.004719,0,0.00%
m1304,EX_m1304,0.0003508,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
m1287,EX_m1287,-0.01145,0,0.00%
m1291,EX_m1291,-0.372,0,0.00%
m1349,EX_m1349,-1.643,0,0.00%


Checking if the exchange reactions is open too and it is.

In [74]:
pichia_methanol.reactions.EX_m1297

Reaction identifier,EX_m1297
Name,EX_m1297
Memory address,0x027872176ba8
Stoichiometry,m1297 <=> Methanol_CH4O <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Normally, you could use `model.medium` to see what compounds are in the medium. Since the model doesn't use the EX_ reactions to define the medium this is not very helpful but at leas we can see all the exchange reactions

In [29]:
pichia_methanol.medium

{'EX_m1282': 1000.0,
 'EX_m1283': 1000.0,
 'EX_m1284': 1000.0,
 'EX_m1285': 1000.0,
 'EX_m1286': 1000.0,
 'EX_m1287': 1000.0,
 'EX_m1288': 1000.0,
 'EX_m1289': 1000.0,
 'EX_m1290': 1000.0,
 'EX_m1291': 1000.0,
 'EX_m1292': 1000.0,
 'EX_m1293': 1000.0,
 'EX_m1294': 1000.0,
 'EX_m1296': 1000.0,
 'EX_m1297': 1000.0,
 'EX_m1298': 1000.0,
 'EX_m1299': 1000.0,
 'EX_m1300': 1000.0,
 'EX_m1301': 1000.0,
 'EX_m1302': 1000.0,
 'EX_m1303': 1000.0,
 'EX_m1304': 1000.0,
 'EX_m1305': 1000.0,
 'EX_m1306': 1000.0,
 'EX_m1307': 1000.0,
 'EX_m1308': 1000.0,
 'EX_m1309': 1000.0,
 'EX_m1310': 1000.0,
 'EX_m1311': 1000.0,
 'EX_m1312': 1000.0,
 'EX_m1313': 1000.0,
 'EX_m1314': 1000.0,
 'EX_m1315': 1000.0,
 'EX_m1316': 1000.0,
 'EX_m1317': 1000.0,
 'EX_m1318': 1000.0,
 'EX_m1319': 1000.0,
 'EX_m1320': 1000.0,
 'EX_m1321': 1000.0,
 'EX_m1322': 1000.0,
 'EX_m1323': 1000.0,
 'EX_m1324': 1000.0,
 'EX_m1325': 1000.0,
 'EX_m1326': 1000.0,
 'EX_m1327': 1000.0,
 'EX_m1328': 1000.0,
 'EX_m1329': 1000.0,
 'EX_m1330': 

You can also get the exchange reactions like this.

In [33]:
pichia_methanol.exchanges

[<Reaction EX_m1282 at 0x1507c4940>,
 <Reaction EX_m1283 at 0x150de6588>,
 <Reaction EX_m1284 at 0x1507c4978>,
 <Reaction EX_m1285 at 0x1507c49b0>,
 <Reaction EX_m1286 at 0x1507c49e8>,
 <Reaction EX_m1287 at 0x1507c4a20>,
 <Reaction EX_m1288 at 0x1507c4a58>,
 <Reaction EX_m1289 at 0x1507c4a90>,
 <Reaction EX_m1290 at 0x1507c4ac8>,
 <Reaction EX_m1291 at 0x1507c4b00>,
 <Reaction EX_m1292 at 0x1507c4b38>,
 <Reaction EX_m1293 at 0x1507c4b70>,
 <Reaction EX_m1294 at 0x1507c4ba8>,
 <Reaction EX_m1296 at 0x1507c4be0>,
 <Reaction EX_m1297 at 0x1507c4c18>,
 <Reaction EX_m1298 at 0x1507c4c50>,
 <Reaction EX_m1299 at 0x1507c4c88>,
 <Reaction EX_m1300 at 0x1507c4cc0>,
 <Reaction EX_m1301 at 0x1507c4cf8>,
 <Reaction EX_m1302 at 0x1507c4d30>,
 <Reaction EX_m1303 at 0x1507c4d68>,
 <Reaction EX_m1304 at 0x1507c4da0>,
 <Reaction EX_m1305 at 0x1507c4dd8>,
 <Reaction EX_m1306 at 0x1507c4e10>,
 <Reaction EX_m1307 at 0x1507c4e48>,
 <Reaction EX_m1308 at 0x1507c4e80>,
 <Reaction EX_m1309 at 0x1507c4eb8>,
 

Let's extract all the compounds associated with them.

In [124]:
exchange2compound = {}
for reaction in pichia_methanol.exchanges:
    exchange2compound[reaction.id] = reaction.reactants[0].name

In [125]:
exchange2compound

{'EX_m1282': '(R)-Lactate_C3H6O3',
 'EX_m1283': '(S)-Lactate_C3H6O3',
 'EX_m1284': 'Acetate_C2H4O2',
 'EX_m1285': '(S)(+)-Allantoin_C4H6N4O3',
 'EX_m1286': 'Biotin_C10H16N2O3S',
 'EX_m1287': 'Biomass_',
 'EX_m1288': 'Carnitine_C7H16NO3',
 'EX_m1289': 'Cellobiose_C12H22O11',
 'EX_m1290': 'Chitin_(C8H13NO5)n',
 'EX_m1291': 'CO2_CO2',
 'EX_m1292': 'Ethanol_C2H6O',
 'EX_m1293': 'alpha-D-Glucose_C6H12O6',
 'EX_m1294': 'Glycerol_C3H8O3',
 'EX_m1296': 'Mannitol_C6H14O6',
 'EX_m1297': 'Methanol_CH4O',
 'EX_m1298': 'Oxygen_O2',
 'EX_m1299': '(9Z)-Octadecenoic acid_C18H34O2',
 'EX_m1300': 'NH3_NH3',
 'EX_m1301': 'Orthophosphate_H3PO4',
 'EX_m1302': 'Protein_C2H4NO2R(C2H2NOR)n',
 'EX_m1303': 'Pyruvate_C3H4O3',
 'EX_m1304': 'Sulfate_H2SO4',
 'EX_m1305': 'D-Sorbitol_C6H14O6',
 'EX_m1306': 'Sulfur_S',
 'EX_m1307': 'Thiamin_C12H17N4OS',
 'EX_m1308': 'alpha,alpha-Trehalose_C12H22O11',
 'EX_m1309': 'Urea_CH4N2O',
 'EX_m1310': 'L-Alanine_C3H7NO2',
 'EX_m1311': 'L-Proline_C5H9NO2',
 'EX_m1312': 'L-Methio

Ok, don't have more time. Let's meet and discuss this if you like.

## Methanol structure used as guide for methane

Now, focus on all methanol reactions to see how to simulate it for the methane uptake.

First we do it for m1297, that is the boundary methanol:

In [117]:
pichia_methanol.metabolites.m1297.reactions

frozenset({<Reaction EX_m1297 at 0x27874bf9eb8>,
           <Reaction r1158 at 0x27872011a58>})

In [121]:
pichia_methanol.reactions.EX_m1297

Reaction identifier,EX_m1297
Name,EX_m1297
Memory address,0x027874bf9eb8
Stoichiometry,m1297 <=> Methanol_CH4O <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [126]:
pichia_methanol.reactions.r1158

Reaction identifier,r1158
Name,uptake of methanol
Memory address,0x0278766b8128
Stoichiometry,m1297 --> m1219 Methanol_CH4O --> Methanol_CH4O
GPR,
Lower bound,0
Upper bound,10


Thus, we see that we start with the boundary one, meaning methanol on the environment near our cell, that gets into extracellular by the uptake, meaning next to the cell membrane on the outside, ready to be introduced.

And it is on this second reaction r1158 that we need to introduce our bounds for uptake.

Now, for m1219, the extracellular:

In [127]:
pichia_methanol.metabolites.m1219.reactions

frozenset({<Reaction r1158 at 0x278766b8128>,
           <Reaction r1382 at 0x278766a0668>})

In [128]:
pichia_methanol.reactions.r1158

Reaction identifier,r1158
Name,uptake of methanol
Memory address,0x0278766b8128
Stoichiometry,m1297 --> m1219 Methanol_CH4O --> Methanol_CH4O
GPR,
Lower bound,0
Upper bound,10


In [129]:
pichia_methanol.reactions.r1382

Reaction identifier,r1382
Name,Methanol permease
Memory address,0x0278766a0668
Stoichiometry,m1219 --> m1215 Methanol_CH4O --> Methanol_CH4O
GPR,
Lower bound,0.0
Upper bound,1000.0


From extracellular we get into m1215 --> cytosol methanol. Let's check its reactions (avoiding to repeat the reaction r1382, already shown above where it is a product):

In [130]:
pichia_methanol.metabolites.m1215.reactions

frozenset({<Reaction r1310 at 0x27876670cc0>,
           <Reaction r1382 at 0x278766a0668>})

In [131]:
pichia_methanol.reactions.r1310

Reaction identifier,r1310
Name,Methanol peroxisomal transport
Memory address,0x027876670cc0
Stoichiometry,m1215 --> m956 Methanol_CH4O --> Methanol_CH4O
GPR,
Lower bound,0.0
Upper bound,1000.0


Which obviously represents the transport to peroxisomes, giving the peroxisomal methanol m956. And it will be there where the MUT pathway will start.

In [133]:
pichia_methanol.metabolites.m956.reactions

frozenset({<Reaction r1310 at 0x27876670cc0>, <Reaction r91 at 0x2787663f8d0>})

In [134]:
pichia_methanol.reactions.r91

Reaction identifier,r91
Name,Methanol:oxygen oxidereductase
Memory address,0x02787663f8d0
Stoichiometry,m711 + m956 --> m713 + m957 Oxygen_O2 + Methanol_CH4O --> H2O2_H2O2 + Formaldehyde_CH2O
GPR,PAS_chr4_0821 or PAS_chr4_0152
Lower bound,0.0
Upper bound,1000.0


Which is already using the methanol to get formaldehyde.

Conclusion: So for methane, we just need to define boundary methane and extracellular methane, becuase once going through the pMMO, it will already enter the cell as methanol.

Questions:

- And then I guess this methanol should be considered as m1215 (cytosolic methanol) that needs to get into the peroxisomes and follow the normal path?

- On M. capsulatus the pMMO reaction occurs at the periplasm, for K. phaffi can we directly consider we change from extracellular to cytosol compartment?

- On M. capsulatus the transformation of Ubiquinol found in the inner membrane into Ubiquinone is also found on the pMMO reaction. However, for the K. phaffii model, those two species are just found on the mitochondria. How should we proceed with that?

- From M. capsulatus model, difference between pMMO and pMMODC (Direct Coupling)

- I guess we should introduce the pMMO gene on the listOfGeneProducts to have our new introduced reaction related to the gene (line 18847) iLC915_pMMO.xml file.

- Also create group for the pMMO reaction? (last part of the file and then all reactions are related to a concrete group when defined)

In [10]:
pichia_pmmo = cobra.io.read_sbml_model("data/iLC915_pMMO.xml")

In [11]:
pichia_pmmo.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m1298,EX_m1298,18.38,0,0.00%
m1300,EX_m1300,0.3301,0,0.00%
m1301,EX_m1301,0.02359,0,0.00%
m1304,EX_m1304,0.001754,0,0.00%
methane_b,EX_methane_b,10,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
m1287,EX_m1287,-0.05727,0,0.00%
m1291,EX_m1291,-6.86,0,0.00%
m1349,EX_m1349,-22.39,0,0.00%


In [15]:
pichia_pmmo.reactions.query("methane", "name")

[<Reaction EX_methane_b at 0x226acc8e4e0>,
 <Reaction methane_uptake at 0x226ad5e7358>]

In [16]:
pichia_pmmo.reactions.methane_uptake

Reaction identifier,methane_uptake
Name,uptake of methane
Memory address,0x0226ad5e7358
Stoichiometry,methane_b --> methane_e methane_b --> methane_e
GPR,
Lower bound,0.0
Upper bound,10.0


Now, it is time to introduce the hemoglobin reaction considering the amino acid and the other things required. The hemoglobin aa sequence is: 

MGAFTEKQEALVSSSFEAFKANIPQYSVVFYTSILEKAPAAKDLFSFLSNGVDPSNPKLTGHAEKLFGLVRDSAGQLKANGTVVADAALGSIHAQKAITDPQFVVVKEALLKTIKEAVGDKWSDELSSAWEVAYDELAAAIKKAF

Checking for biomass reactions, because some of them take the amino acid as reactants, thus we are interested in those for the protein production:

In [7]:
pichia.reactions.query("biomass", "name")

[<Reaction r1133 at 0x1ed67bd8e80>]

In [8]:
pichia.reactions.query("Biomass", "name")

[<Reaction r1090 at 0x1ed67c3eb38>, <Reaction r1187 at 0x1ed67c8b128>]

In [15]:
print(pichia.reactions.r1133)
print('r91', pichia.reactions.r1133.name, ':', pichia.metabolites.m1226.name, '-->',  pichia.metabolites.m1287.name)

r1133: m1226 --> m1287
r91 Drain of biomass : Biomass_ --> Biomass_


In [18]:
print(pichia.reactions.r1090)
print('r1090',  pichia.reactions.r1090.name, ':', pichia.metabolites.m1.name, '+', pichia.metabolites.m1336.name, '+', pichia.metabolites.m217.name, '+', pichia.metabolites.m373.name, '+', pichia.metabolites.m379.name, '+', pichia.metabolites.m5.name, '-->',  pichia.metabolites.m1225.name, '+', pichia.metabolites.m3.name, '+', pichia.metabolites.m7.name)

r1090: 35.7 m1 + 0.369 m1336 + 0.0756 m1337 + 0.37 m217 + 0.0583 m373 + 0.0013 m379 + 35.7 m5 --> m1225 + 35.7 m3 + 35.7 m7
r1090 Biomass (g/gDW) : ATP_C10H16N5O13P3 + Carbohydrates_ + Protein_C2H4NO2R(C2H2NOR)n + RNA_C10H18O13P2R2(C5H8O6PR)n + DNA_C10H17O8PR2(C5H8O5PR)n + H2O_H2O --> Biomass_ + ADP_C10H15N5O10P2 + Orthophosphate_H3PO4


In [14]:
print(pichia.reactions.r1187)

r1187: 20.7 m1 + 0.002 m1115 + 0.0388 m1117 + 0.0007 m1119 + 0.0007 m1121 + 0.0002 m1123 + 0.33208 m153 + 0.2689 m154 + 0.15382 m156 + 0.19585 m158 + 0.15382 m161 + 0.2689 m163 + 0.040222 m177 + 0.13623 m185 + 0.22237 m203 + 0.24035 m210 + 0.20297 m222 + 0.24287 m227 + 0.14544 m239 + 1.701 m26 + 0.10532 m272 + 0.074636 m274 + 0.22018 m279 + 0.0096 m28 + 0.046031 m280 + 0.19541 m305 + 0.1689 m31 + 0.0057539 m331 + 0.024879 m343 + 0.051 m384 + 0.00082 m389 + 0.00118 m404 + 0.067 m418 + 0.05 m423 + 0.00082 m431 + 0.00118 m437 + 0.20522 m442 + 0.05984 m490 + 20.7 m5 + 0.16654 m625 + 0.03292 m669 + 0.05956 m672 + 0.33054 m675 + 0.20522 m678 + 0.001 m704 + 0.001 m821 + 0.002 m825 + 0.064 m827 + 0.116 m838 + 0.003 m846 + 0.051 m94 --> m1225 + 20.7 m3 + 20.7 m7
r1382 Methanol permease : Methanol_CH4O --> Methanol_CH4O


In [24]:
print('r1382',  pichia.reactions.r1382.name, ':', pichia.metabolites.m1.name, '+', pichia.metabolites.m1115.name, '+', pichia.metabolites.m1117.name, '+', pichia.metabolites.m1119.name, '+', pichia.metabolites.m1121.name, '+', pichia.metabolites.m1123.name, pichia.metabolites.m153.name, '+' , pichia.metabolites.m154.name, '+' ,
      pichia.metabolites.m156.name, '+' , pichia.metabolites.m158.name, '+' , pichia.metabolites.m161.name, '+' , pichia.metabolites.m163.name, '+',  pichia.metabolites.m177.name, '+', pichia.metabolites.m185.name, '+', pichia.metabolites.m203.name, '+', pichia.metabolites.m210.name, '+', pichia.metabolites.m222.name, '+', 
      pichia.metabolites.m227.name, '+', pichia.metabolites.m239.name, '+' , pichia.metabolites.m26.name, '+' , pichia.metabolites.m272.name, '+', pichia.metabolites.m274.name, '+', pichia.metabolites.m279.name, '+', pichia.metabolites.m28.name, '+', pichia.metabolites.m280.name, '+', pichia.metabolites.m1.name, '+', pichia.metabolites.m305.name, '+', 
      pichia.metabolites.m31.name, '+', pichia.metabolites.m331.name, '+', pichia.metabolites.m343.name, '+', pichia.metabolites.m384.name, '+', pichia.metabolites.m389.name, '+', pichia.metabolites.m404.name, pichia.metabolites.m418.name, '+' , pichia.metabolites.m423.name, '+' ,
      pichia.metabolites.m431.name, '+' , pichia.metabolites.m437.name, '+' , pichia.metabolites.m442.name, '+' , pichia.metabolites.m490.name, '+', pichia.metabolites.m5.name, '+', pichia.metabolites.m625.name, '+', pichia.metabolites.m669.name, '+', pichia.metabolites.m672.name, '+', pichia.metabolites.m675.name, '+', 
      pichia.metabolites.m678.name, '+', pichia.metabolites.m704.name, '+' , pichia.metabolites.m821.name, '+' , pichia.metabolites.m825.name, '+', pichia.metabolites.m827.name, '+', pichia.metabolites.m838.name, '+', pichia.metabolites.m846.name, '+', pichia.metabolites.m94.name, '-->',  pichia.metabolites.m1225.name, '+', pichia.metabolites.m3.name, '+', pichia.metabolites.m7.name)

r1382 Methanol permease : ATP_C10H16N5O13P3 + Zymosterol_C27H44O + Ergosterol_C28H44O + Episterol_C28H46O + Fecosterol_C28H46O + Lanosterol_C30H50O L-Alanine_C3H7NO2 + L-Glutamate_C5H9NO4 + L-Aspartate_C4H7NO4 + L-Arginine_C6H14N4O2 + L-Asparagine_C4H8N2O3 + L-Glutamine_C5H10N2O3 + L-Ornithine_C5H12N2O2 + L-Proline_C5H9NO2 + L-Lysine_C6H14N2O2 + Glycine_C2H5NO2 + L-Valine_C5H11NO2 + L-Leucine_C6H13NO2 + L-Isoleucine_C6H13NO2 + 1,3-beta-D-Glucan_(C6H10O5)n + L-Phenylalanine_C9H11NO2 + L-Tyrosine_C9H11NO3 + L-Serine_C3H7NO3 + alpha,alpha-Trehalose_C12H22O11 + L-Tryptophan_C11H12N2O2 + ATP_C10H16N5O13P3 + L-Threonine_C4H9NO3 + Glycogen_C24H42O21 + L-Cysteine_C3H7NO2S + L-Methionine_C5H11NO2S + GMP_C10H14N5O8P + dGMP_C10H14N5O7P + dAMP_C10H14N5O6P UMP_C9H13N2O9P + CMP_C9H14N3O8P + dCMP_C9H14N3O7P + dTMP_C10H15N2O8P + (9Z,12Z,15Z)-Octadecatrienoic acid_C18H30O2 + L-Histidine_C6H9N3O2 + H2O_H2O + Hexadecanoic acid_C16H32O2 + Octadecanoic acid_C18H36O2 + (9Z)-Hexadecenoic acid_C16H30O2 + (9Z)

In [3]:
pichia.reactions.query("FAB", "name")

[<Reaction r1338 at 0x1aa84028320>,
 <Reaction r1337 at 0x1aa840a7128>,
 <Reaction r1100 at 0x1aa840d0198>,
 <Reaction r1101 at 0x1aa840c8e10>,
 <Reaction r1102 at 0x1aa840f13c8>,
 <Reaction r1103 at 0x1aa840f1978>]

In [5]:
print(pichia.reactions.r1100, pichia.reactions.r1101, pichia.reactions.r1102, pichia.reactions.r1103, pichia.reactions.r1337, pichia.reactions.r1338)

r1100: 39.3881 m1 + 0.5591 m153 + 0.4731 m154 + 0.387 m156 + 0.43 m158 + 0.215 m161 + 0.55901 m163 + 0.5161 m185 + 0.387 m203 + 0.6021 m210 + 0.6451 m222 + 0.7741 m227 + 0.215 m239 + 0.387 m272 + 0.387 m274 + 1.3761 m279 + 0.086 m280 + 0.7311 m305 + 0.215 m331 + 0.043 m343 + 0.172 m490 + 39.3881 m5 --> m1360 + 39.3881 m3 + 39.3881 m7 r1101: 10.447 m1 + 0.569 m389 + 1.182 m404 + 0.584 m431 + 0.737 m437 + 10.447 m5 --> m1362 + 10.447 m3 + 10.447 m7 r1102: 0.997 m1360 + 0.0029 m1361 + 2.8e-05 m1362 --> m1363 r1103: 7.374 m1 + 0.584 m384 + 1.182 m418 + 0.569 m423 + 7.374 m5 + 0.737 m94 --> m1361 + 7.374 m3 + 7.374 m7 r1337: m1364 --> m1365 r1338: m1363 --> m1364


Better to check on the sbml file directly